# Project Two Dashboard (Grazioso Salvare)
_Generated scaffold on 2025-10-14T16:27:19.007309Z_

This notebook creates the Dash dashboard with:
- Interactive filters (rescue type)
- Data table that updates from MongoDB via your CRUD module
- Geolocation map (dash-leaflet) and a bar chart

> **Remember**: Update your Mongo credentials if needed and verify breed mappings with the Dashboard Specifications Document.

In [ ]:
# --- Imports
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, dash_table, Input, Output, State, callback_context
import dash_leaflet as dl
import pandas as pd
import plotly.express as px
from AnimalShelter import AnimalShelter

# --- Connect to MongoDB via CRUD module
USERNAME = "aacuser"     # update if needed
PASSWORD = "SNHU1234"    # update if needed
HOST     = "localhost"
PORT     = 27017         # 27017 (Codio default) or 27018 if you changed it

shelter = AnimalShelter(username=USERNAME, password=PASSWORD, host=HOST, port=PORT)

# --- Helper: pull data with optional query
def fetch_dataframe(query: dict | None=None, limit: int=500):
    query = query or {}
    docs = shelter.read_all(query, limit=limit)
    if not docs:
        return pd.DataFrame()
    df = pd.DataFrame(docs)
    # Tidy up for display
    if "_id" in df.columns:
        df["_id"] = df["_id"].astype(str)
    return df


In [ ]:
# --- Build base (unfiltered) dataframe
base_df = fetch_dataframe({}, limit=500)

# --- Build the app
app = JupyterDash(__name__)

logo_path = "/mnt/data/CS 340_Project Two_Module Seven Introduction_Branded Image.jpg"  # copied into this environment
identifier = "Dashboard by Raziuddin Syed"   # <-- change to your name as required

app.layout = html.Div([

    html.Div([

        html.Img(src=logo_path, style={"maxWidth":"280px","height":"auto","marginRight":"16px"}),

        html.H2("Grazioso Salvare Candidate Search Dashboard"),

        html.P(identifier)

    ], style={"display":"flex","alignItems":"center","gap":"12px"}),


    html.Hr(),


    html.Div([

        html.Label("Rescue Type:"),

        dcc.RadioItems(

            id="rescue-type",

            options=[

                {"label":"Reset (All)", "value":"reset"},

                {"label":"Water Rescue", "value":"water"},

                {"label":"Mountain/Wilderness Rescue", "value":"mountain_wilderness"},

                {"label":"Disaster/Individual Tracking", "value":"disaster_individual"}

            ],

            value="reset",

            inline=True

        )

    ], style={"padding":"8px 0"}),


    dash_table.DataTable(

        id="animals-table",

        columns=[{{"name": c, "id": c}} for c in (base_df.columns if not base_df.empty else [])],

        data=(base_df.to_dict("records") if not base_df.empty else []),

        page_size=10,

        filter_action="native",

        sort_action="native",

        style_table={{"overflowX":"auto"}},

    ),


    html.Br(),

    html.Div([

        html.Div([

            html.H4("Geolocation Map"),

            dl.Map(center=[30.27, -97.74], zoom=9, children=[

                dl.TileLayer(),

                dl.LayerGroup(id="marker-layer")

            ], style={{"width":"100%","height":"430px"}}),

        ], style={{"flex":"1"}}),

        html.Div([

            html.H4("Breed Distribution"),

            dcc.Graph(id="breed-chart")

        ], style={{"flex":"1"}}),

    ], style={{"display":"flex","gap":"16px"}}),


])



In [ ]:
# --- Callbacks: update table, map, and chart based on rescue type selection
@app.callback(

    Output("animals-table","data"),

    Output("animals-table","columns"),

    Output("marker-layer","children"),

    Output("breed-chart","figure"),

    Input("rescue-type","value")

)

def on_rescue_type_change(rescue):

    if rescue == "reset":

        query = {}

    else:

        query = shelter.query_for_rescue_type(rescue)


    df = fetch_dataframe(query, limit=500)

    if df.empty:

        return [], [], [], px.scatter(title="No data")


    # --- Table

    columns = [{"name": c, "id": c} for c in df.columns]

    data = df.to_dict("records")


    # --- Map markers (expects lat/long columns as per spec)

    markers = []

    lat_col = "location_lat"

    lon_col = "location_long"

    if lat_col in df.columns and lon_col in df.columns:

        for _, row in df.dropna(subset=[lat_col, lon_col]).iterrows():

            markers.append(dl.Marker(position=[row[lat_col], row[lon_col]], children=dl.Tooltip(str(row.get("breed","Dog")))))


    # --- Breed bar chart

    if "breed" in df.columns:

        fig = px.bar(df["breed"].value_counts().reset_index(), x="index", y="breed", labels={"index":"Breed","breed":"Count"}, title="Breed Distribution")

    else:

        fig = px.bar(title="Breed Distribution")


    return data, columns, markers, fig



In [ ]:
# --- Run the app (inside Jupyter)

app.run_server(mode="inline", port=8050, dev_tools_ui=True, debug=False)

